In [10]:
import asyncio
import websockets
import pandas as pd

In [21]:
def make_dataframe(url, index_name):
    #takes url for given deribit method we want to get and makes into a data frame
    #we do this in the notebook since jupyter takes up a thread for asyncio and 
    #I don't know how to fix that by using multithreaded stuff yet
    with urllib.request.urlopen(url) as url:
        data = json.loads(url.read().decode())
    data = pd.DataFrame(data['result']).set_index(index_name)
    return data

In [23]:
make_dataframe("https://test.deribit.com/api/v2/public/get_book_summary_by_currency?currency=BTC&kind=future", 'instrument_name')

,volume_usd,volume,quote_currency,price_change,open_interest,mid_price,mark_price,low,last,high,estimated_delivery_price,creation_timestamp,bid_price,base_currency,ask_price,funding_8h,current_funding
instrument_name,,,,,,,,,,,,,,,,,
BTC-25JUN21,1.827708e+07,503.52,USD,-3.7605,46973090,36956.50,36950.92,35174.0,36942.0,38385.5,34880.32,1610895827423,36948.0,BTC,36965.0,NaN,NaN
BTC-26MAR21,2.856233e+07,791.62,USD,-8.1245,295570690,36203.75,36210.26,34297.0,36204.0,39518.0,34880.32,1610895827423,36195.5,BTC,36212.0,NaN,NaN
BTC-24SEP21,1.448887e+07,373.50,USD,-6.6842,84602270,37768.50,37781.05,37650.0,38119.5,41363.0,34880.32,1610895827423,37743.5,BTC,37793.5,NaN,NaN
BTC-PERPETUAL,1.715352e+09,47877.92,USD,-6.4234,1853980552,34887.75,34887.53,33495.5,34883.5,38328.5,34880.32,1610895827423,34887.0,BTC,34888.5,0.000021,0.0
